In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt  

In [3]:
torch.manual_seed(10)

In [4]:
df = pd.read_csv('./datasets/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [5]:
X = df.drop(columns=['label'])
y = df['label']

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [8]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features.values,dtype=torch.float32)
        self.labels = torch.tensor(labels.values,dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        return self.features[index],self.labels[index]        

In [9]:
train_dataset = CustomDataset(X_train,y_train)

In [10]:
len(train_dataset)

4800

In [11]:
test_dataset = CustomDataset(X_test,y_test)

In [12]:
len(X_test)

1200

In [13]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [14]:
class NN(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
        
    def forward(self, x):
        return self.network(x)        

In [15]:
epochs = 100
learning_rate = 0.01

In [16]:
# instantiate the model
model = NN(X_train.shape[1])

# loss function
loss_function = nn.CrossEntropyLoss()
 
# optimizer 
optimizer = optim.SGD(model.parameters(),learning_rate)    

In [17]:
for epoch in range(epochs):
    epoch_loss = 0
    for batch_features,batch_labels in train_loader:
        # forward pass
        y_pred = model(batch_features)
        
        # calculate loss
        loss = loss_function(y_pred,batch_labels)
        
        # Back pass
        optimizer.zero_grad()
        loss.backward() 
        
        # update parameters
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch}: Loss: {epoch_loss}")        
    
avg_loss = epoch_loss / len(train_loader)     

Epoch 0: Loss: 329.8446595668793
Epoch 1: Loss: 277.8671337366104
Epoch 2: Loss: 212.282160282135
Epoch 3: Loss: 172.0891523361206
Epoch 4: Loss: 148.81258696317673
Epoch 5: Loss: 134.3056784272194
Epoch 6: Loss: 125.15527987480164
Epoch 7: Loss: 118.77223527431488
Epoch 8: Loss: 113.3487634062767
Epoch 9: Loss: 108.94896304607391
Epoch 10: Loss: 105.4925326704979
Epoch 11: Loss: 101.95368853211403
Epoch 12: Loss: 99.27613332867622
Epoch 13: Loss: 96.3955186009407
Epoch 14: Loss: 93.11757242679596
Epoch 15: Loss: 91.22387054562569
Epoch 16: Loss: 89.01160556077957
Epoch 17: Loss: 87.0930772125721
Epoch 18: Loss: 85.5559822767973
Epoch 19: Loss: 84.42306166887283
Epoch 20: Loss: 82.18166156113148
Epoch 21: Loss: 80.76551994681358
Epoch 22: Loss: 79.42049044370651
Epoch 23: Loss: 78.13504467904568
Epoch 24: Loss: 76.92593061923981
Epoch 25: Loss: 75.96466994285583
Epoch 26: Loss: 75.04027150571346
Epoch 27: Loss: 73.86805474758148
Epoch 28: Loss: 72.96005533635616
Epoch 29: Loss: 71.6223

In [18]:
model.eval()

NN(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [19]:
total = 0
correct = 0

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total = total + batch_labels.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()
    
    print(correct/total)        

0.84
